In [ ]:
import numpy as np 
import pandas as pd 
 
import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import tensorflow as tf
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [ ]:
# lines=pd.read_csv("/content/drive/MyDrive/Hindi_English_Truncated_Corpus.csv",encoding='utf-8')
url = 'https://raw.githubusercontent.com/prismspeechproject/neural/master/implementation/English_hindi.csv'
lines=pd.read_csv(url, encoding='utf-8')
lines= lines[lines.columns[0:2]]
lines=lines[~pd.isnull(lines['English'])]
lines=lines[~pd.isnull(lines['Hindi'])]
lines.drop_duplicates(inplace=True)
lines=lines.sample(n=25000,random_state=42)
lines.shape

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,8,9,10,11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(25000, 2)

In [ ]:
lines['English']=lines['English'].apply(lambda x: x.lower())
lines['Hindi']=lines['Hindi'].apply(lambda x: x.lower())

In [ ]:

lines['English']=lines['English'].apply(lambda x: re.sub("'", '', x))
lines['Hindi']=lines['Hindi'].apply(lambda x: re.sub("'", '', x))

In [ ]:
exclude = set(string.punctuation) 
lines['English']=lines['English'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['Hindi']=lines['Hindi'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [ ]:
remove_digits = str.maketrans('', '', digits)
lines['English']=lines['English'].apply(lambda x: x.translate(remove_digits))
lines['Hindi']=lines['Hindi'].apply(lambda x: x.translate(remove_digits))

lines['Hindi'] = lines['Hindi'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

lines['English']=lines['English'].apply(lambda x: x.strip())
lines['Hindi']=lines['Hindi'].apply(lambda x: x.strip())
lines['English']=lines['English'].apply(lambda x: re.sub(" +", " ", x))
lines['Hindi']=lines['Hindi'].apply(lambda x: re.sub(" +", " ", x))

In [ ]:

lines['Hindi'] = lines['Hindi'].apply(lambda x : 'START_ '+ x + ' _END')
lines.head()

,English,Hindi
93994,it is the land of festivals of different religions like diwali dushehra pongal and onam idulfitr muhrram chrimas ester are very much famous,START_ विभिन्न धर्मों के इस भूभाग पर कई मनभावन पर्व त्यौहार मनाए जाते हैं दिवाली होली दशहरा पोंगल तथा ओणम ईदउलफितर मुहर्रम क्रिसमस ईस्टर आदि भी काफ़ी लोकप्रिय हैं। _END
124301,enceladus seen here,START_ एनसेलेडस पर इस जगह _END
28869,ghanaian businessmen south african enterprising leaders,START_ या फिर घाना के व्यवसायियों से या दक्षिण अफ्रिका के उद्योगों में अग्रणी भूमिका निभाने वालों से _END
54938,the burma oil company processed the crude in its refinery at rangoon and distributed the products mainly kerosene through its tank installations at chief indian ports and through a fleet of steam tankers,START_ बर्मा आयल कंपनी अपनी रंगून की रिफाइनरी में कच्चे तेल को साफ करती थी और अपने उत्पादन का वितरण जो मुख़्यत मिट्टी का तेल था प्रधान भारतीय बंदरगाहों पर लगे टैंक प्रतिष्ठानों के द्वारा तथा स्टीम टैंकरों के द्वारा करती थी _END
49997,ismail afandi aka ismail khan of the ottoman empire designer of the main dome,START_ मुख्य गुम्बद का अभिकल्पक इस्माइल एकाइस्माइल खाँ जो कि ऑट्टोमन साम्राज्य का प्रमुख गोलार्ध एवं गुम्बद अभिकल्पक थे। _END


In [ ]:

all_eng_words=set()
for eng in lines['English']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['Hindi']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [ ]:
len(all_eng_words)

30829

In [ ]:
len(all_hindi_words)

37434

In [ ]:
lines['length_eng_sentence']=lines['English'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['Hindi'].apply(lambda x:len(x.split(" ")))

In [ ]:
lines.head()

,English,Hindi,length_eng_sentence,length_hin_sentence
93994,it is the land of festivals of different religions like diwali dushehra pongal and onam idulfitr muhrram chrimas ester are very much famous,START_ विभिन्न धर्मों के इस भूभाग पर कई मनभावन पर्व त्यौहार मनाए जाते हैं दिवाली होली दशहरा पोंगल तथा ओणम ईदउलफितर मुहर्रम क्रिसमस ईस्टर आदि भी काफ़ी लोकप्रिय हैं। _END,23,30
124301,enceladus seen here,START_ एनसेलेडस पर इस जगह _END,3,6
28869,ghanaian businessmen south african enterprising leaders,START_ या फिर घाना के व्यवसायियों से या दक्षिण अफ्रिका के उद्योगों में अग्रणी भूमिका निभाने वालों से _END,6,19
54938,the burma oil company processed the crude in its refinery at rangoon and distributed the products mainly kerosene through its tank installations at chief indian ports and through a fleet of steam tankers,START_ बर्मा आयल कंपनी अपनी रंगून की रिफाइनरी में कच्चे तेल को साफ करती थी और अपने उत्पादन का वितरण जो मुख़्यत मिट्टी का तेल था प्रधान भारतीय बंदरगाहों पर लगे टैंक प्रतिष्ठानों के द्वारा तथा स्टीम टैंकरों के द्वारा करती थी _END,33,43
49997,ismail afandi aka ismail khan of the ottoman empire designer of the main dome,START_ मुख्य गुम्बद का अभिकल्पक इस्माइल एकाइस्माइल खाँ जो कि ऑट्टोमन साम्राज्य का प्रमुख गोलार्ध एवं गुम्बद अभिकल्पक थे। _END,14,20


In [ ]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]

In [ ]:
print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [ ]:
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

In [ ]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(30829, 37434)

In [ ]:
num_decoder_tokens += 1 
num_encoder_tokens = num_encoder_tokens + 1 

In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [ ]:
lines = shuffle(lines)
lines.head(10)

,English,Hindi,length_eng_sentence,length_hin_sentence
66783,something that is absent in indopak engagement,START_ और भारतपाक वार्ता के बीच यही चीज नदारद है _END,7,11
85524,these islands came under the administration of arzi hukumateazad hind the provisional government of free india,START_ यह द्वीप अर्जीहुकुमतएआजादहिंद के अनुशासन में रहें। _END,16,9
9925,we need to face our fears and take back our streets,START_ हमें डर का सामना करना होगा और वापस अपनी सड़कों पर जाना होगा _END,11,15
110753,the guess was percent which is,START_ अनुमान है कि प्रतिशत है जो है था _END,6,10
94381,started as a teenager,START_ एक किशोर बालिका के रूप में शुरू हुई _END,4,10
105623,this is sweet and full of juices,START_ यह मधुर और सरस है। _END,7,7
9510,and i think that had something to do with giving me the courage,START_ मुझे लगता है कि कहीं तो इस बात ने मुझे हिम्मत भी जुगाई कि _END,13,16
125265,due to oregon treaty with britain america got control over present american north west,START_ में ब्रिटेन के साथ हुई ओरेगॉन संधि के कारण अमेरिका को वर्तमान अमेरिकी उत्तरपश्चिम पर नियंत्रण मिला। _END,14,19
31347,into areas of life that are not going too well,START_ पहलुओं के बारे में काफ़ी ज्ञान मिलेगा _END,10,9
26524,“ here comes gautama our old teacher ” observed one of them,START_ उनमें से एक ने कहा देखो वह आ रहा है गौतम हमारा शिक्षक _END,12,15


In [ ]:
X, y = lines['English'], lines['Hindi']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((13168,), (3293,))

In [ ]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [ ]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [ ]:
latent_dim=300
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [ ]:
train_samples = len(X_train)
val_samples = len(X_test)
epochs = 100
batch_size =128
# checkpoint_path = "/content/drive/MyDrive/Colab/ResultPNBy/" /training_2/cp-{epoch:04d}.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# cp_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_path, 
#     verbose=1, 
#     save_weights_only=True,
#     save_freq=5*batch_size)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()


model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples/batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples/batch_size)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 300)    9249000     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    11230500    input_2[0][0]                    
______________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
102/102 [==============================] - 66s 410ms/step - loss: 4.0690 - val_loss: 3.5805
Epoch 2/100
102/102 [==============================] - 40s 391ms/step - loss: 3.4631 - val_loss: 3.5108
Epoch 3/100
102/102 [==============================] - 44s 431ms/step - loss: 3.3284 - val_loss: 3.4123
Epoch 4/100
102/102 [==============================] - 44s 432ms/step - loss: 3.1836 - val_loss: 3.3530
Epoch 5/100
102/102 [==============================] - 41s 396ms/step - loss: 3.0660 - val_loss: 3.3135
Epoch 6/100
102/102 [==============================] - 45s 440ms/step - loss: 2.9637 - val_loss: 3.2901
Epoch 7/100
102/102 [==============================] - 41s 397ms/step - loss: 2.8710 - val_loss: 3.2671
Epoch 8/100
102/102 [==============================] - 40s 393ms/step - loss: 2.7832 - val_loss: 3.2579
Epoch 9/100
102/102 [==============================] - 44s 431ms/step - loss: 2.6987 - val_loss: 3.2420
Epoch 10/100
102/102 [==============================] - 40s 387m

KeyboardInterrupt: ignored

In [ ]:
import keras
model.save('/content/drive/MyDrive/Colab/ResultPNA2.h5')
# model = keras.models.load_model('/content/drive/MyDrive/ColabNotebooks/ResultPN.h5')
new_model = keras.models.load_model('/content/drive/MyDrive/Colab/ResultPNA2.h5')

In [ ]:
model.save_weights('/content/drive/MyDrive/Colab/ResultPNA2Weights.h5')

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) 

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
# new_model.get_weights()
decoder_model.save_weights('/content/drive/MyDrive/Colab/ResultPNA2DecoderModelW.h5')
# decoder_model.save('/content/drive/MyDrive/Colab/ResultPNA2DecoderModel.h5')
encoder_model.save_weights('/content/drive/MyDrive/Colab/ResultPNA2EncoderModelW.h5')

# import keras
# encoder1_model = keras.loa('/content/drive/MyDrive/Colab/ResultPNA2EncoderModelW.h5')
# decoder1_model = keras.Model.load_weights(new_model, filepath='/content/drive/MyDrive/Colab/ResultPNA2DecoderModelW.h5')

In [ ]:
encoder_model.get_weights()

[array([[-0.03278746, -0.04713671,  0.04531088, ...,  0.03735603,
          0.04260695, -0.03743915],
        [-0.04260749, -0.04618214,  0.06512512, ...,  0.03285646,
         -0.16743907,  0.17625006],
        [-0.0256097 ,  0.0101689 , -0.02973282, ...,  0.06225212,
          0.00446727,  0.06319299],
        ...,
        [ 0.09077454, -0.15649116,  0.01506825, ..., -0.03724433,
          0.06550077,  0.13268104],
        [ 0.02333522,  0.09466827, -0.065641  , ...,  0.07773712,
         -0.0386913 ,  0.12254772],
        [-0.09309627, -0.00038396,  0.04899064, ...,  0.02046278,
          0.05504965, -0.02658486]], dtype=float32),
 array([[-0.04894491,  0.43890646,  0.14557324, ..., -0.2037875 ,
         -0.05536366,  0.04763384],
        [-0.23519029, -0.28390932, -0.06006496, ..., -0.26280534,
         -0.00966674, -0.04627035],
        [-0.13937916,  0.06998777, -0.21449183, ...,  0.01371102,
         -0.09854786, -0.03401583],
        ...,
        [-0.10170906, -0.3826086 , -0.0

In [ ]:
decoder1_model.load_weights('/content/drive/MyDrive/Colab/ResultPNA2DecoderModelW.h5')
encoder1_model.load_weights('/content/drive/MyDrive/Colab/ResultPNA2EncoderModelW.h5')


In [ ]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = target_token_index['START_']
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: bhabruvahan son of arjun and chitrangadha
Actual Hindi Translation:  बभ्रुवाहन अर्जुन एवं चित्रांग्दा का पुत्र। 
Predicted Hindi Translation:  बभ्रुवाहन अर्जुन एवं चित्रांग्दा का पुत्र। 


In [ ]:
# type(decoded_sentence.split())
y =y_train[k:k+1].values[0][6: -4].split()
Y= decoded_sentence[:-4].split()
# print(type(y), type(Y))
print('BLEU-1: %f' % corpus_bleu(y, Y, weights=(1.0, 0, 0, 0)))
print('BLEU-2: %f' % corpus_bleu(y, Y, weights=(0.5, 0.5, 0, 0)))
print('BLEU-3: %f' % corpus_bleu(y, Y, weights=(0.33, 0.33, 0.33, 0)))
print('BLEU-4: %f' % corpus_bleu(y, Y, weights=(0.25, 0.25, 0.25, 0.25)))
# type(y_train.to_list())

BLEU-1: 0.945946
BLEU-2: 0.972598
BLEU-3: 0.981829
BLEU-4: 0.986204


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
help(corpus_bleu)

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: in india there were million pigs in
Actual Hindi Translation:  भारत में में लाख सूअर थे 
Predicted Hindi Translation:  भारत में में लाख सूअर थे 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: today this dish has become the alternative meaning of indian dishes abroad
Actual Hindi Translation:  आज तो यूरोपियन देशों में करी इंडियन डिशेस का पर्याय बन गया है। 
Predicted Hindi Translation:  आज तक तो प्रेमचंद की जनसंख्या का महत्वपूर्ण हि


In [ ]:
print('Individual 1-gram: %f' % corpus_bleu(y, Y))
print('Individual 2-gram: %f' % corpus_bleu(y, Y))
print('Individual 3-gram: %f' % corpus_bleu(y, Y))
print('Individual 4-gram: %f' % corpus_bleu(y, Y))

Individual 1-gram: 0.993527
Individual 2-gram: 0.993527
Individual 3-gram: 0.993527
Individual 4-gram: 0.993527


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
#urlTest2 = 'https://raw.githubusercontent.com/prismspeechproject/neural/testing/implementation/TestDataset/WordLen3.csv'
word3Df =lines[lines['length_eng_sentence']==3]
# lines=lines[lines['length_hin_sentence']<=20]
x_word3Df = word3Df['English']
y_word3Df = word3Df['Hindi']

In [ ]:
word_gen = generate_batch(x_word3Df, y_word3Df, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(word_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', x_word3Df[k:k+1].values[0])
print('Actual Hindi Translation:', y_word3Df[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])
y =y_word3Df[k:k+1].values[0][6: -4].split()
Y= decoded_sentence[:-4].split()

Input English sentence: languages contain patterns
Actual Hindi Translation:  भाषाओं में पैटर्न होते हैं। 
Predicted Hindi Translation:  भाषाओं में पैटर्न होते हैं। 


In [ ]:
print('BLEU-1: %f' % corpus_bleu(y, Y, weights=(1.0, 0, 0, 0)))
print('BLEU-2: %f' % corpus_bleu(y, Y, weights=(0.5, 0.5, 0, 0)))
print('BLEU-3: %f' % corpus_bleu(y, Y, weights=(0.33, 0.33, 0.33, 0)))
print('BLEU-4: %f' % corpus_bleu(y, Y, weights=(0.25, 0.25, 0.25, 0.25)))
######  3 word result ########### 

BLEU-1: 0.956522
BLEU-2: 0.978019
BLEU-3: 0.985438
BLEU-4: 0.988949


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
word4Df =lines[lines['length_eng_sentence']==4]
# lines=lines[lines['length_hin_sentence']<=20]
x_word4Df = word4Df['English']
y_word4Df = word4Df['Hindi']

In [ ]:
word4_gen = generate_batch(x_word4Df, y_word4Df, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(word4_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', x_word4Df[k:k+1].values[0])
print('Actual Hindi Translation:', y_word4Df[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])
y =y_word4Df[k:k+1].values[0][6: -4].split()
Y= decoded_sentence[:-4].split()

Input English sentence: started as a teenager
Actual Hindi Translation:  एक किशोर बालिका के रूप में शुरू हुई 
Predicted Hindi Translation:  एक किशोर बालिका के रूप में शुरू हुई 


In [ ]:
print('BLEU-1: %f' % corpus_bleu(y, Y, weights=(1.0, 0, 0, 0)))
print('BLEU-2: %f' % corpus_bleu(y, Y, weights=(0.5, 0.5, 0, 0)))
print('BLEU-3: %f' % corpus_bleu(y, Y, weights=(0.33, 0.33, 0.33, 0)))
print('BLEU-4: %f' % corpus_bleu(y, Y, weights=(0.25, 0.25, 0.25, 0.25)))
######  4 word result ########### 

BLEU-1: 0.964286
BLEU-2: 0.981981
BLEU-3: 0.988070
BLEU-4: 0.990949


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
word5Df =lines[lines['length_eng_sentence']==5]
# lines=lines[lines['length_hin_sentence']<=20]
x_word5Df = word5Df['English']
y_word5Df = word5Df['Hindi']

In [ ]:
word5_gen = generate_batch(x_word5Df, y_word5Df, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(word5_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', x_word5Df[k:k+1].values[0])
print('Actual Hindi Translation:', y_word5Df[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])
y =y_word5Df[k:k+1].values[0][6: -4].split()
Y= decoded_sentence[:-4].split()

Input English sentence: every politician in every country
Actual Hindi Translation:  हर देश के राजनेता 
Predicted Hindi Translation:  हर देश के राजनेता 


In [ ]:
print('BLEU-1: %f' % corpus_bleu(y, Y, weights=(1.0, 0, 0, 0)))
print('BLEU-2: %f' % corpus_bleu(y, Y, weights=(0.5, 0.5, 0, 0)))
print('BLEU-3: %f' % corpus_bleu(y, Y, weights=(0.33, 0.33, 0.33, 0)))
print('BLEU-4: %f' % corpus_bleu(y, Y, weights=(0.25, 0.25, 0.25, 0.25)))
######  5 word result ########### 

BLEU-1: 0.928571
BLEU-2: 0.963624
BLEU-3: 0.975841
BLEU-4: 0.981644


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
print('BLEU-1: %f' % corpus_bleu(y, Y, weights=(1.0, 0, 0, 0)))
print('BLEU-2: %f' % corpus_bleu(y, Y, weights=(0.5, 0.5, 0, 0)))
print('BLEU-3: %f' % corpus_bleu(y, Y, weights=(0.33, 0.33, 0.33, 0)))
print('BLEU-4: %f' % corpus_bleu(y, Y, weights=(0.25, 0.25, 0.25, 0.25)))

BLEU-1: 0.928571
BLEU-2: 0.963624
BLEU-3: 0.975841
BLEU-4: 0.981644


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
